# Get Data
The following code block allows a user to get data files and utility scripts beginning with only main.ipynb either locally or on Google Colab.

In [ ]:
import os.path
from os import path
try:
    __import__("wget")
    import wget
    print("wget import exists")
    if(not path.exists("data")):
        print("Making Directory data")
        os.mkdir("data")   
    if(not path.exists("data/control.xls")):
        print("Downloading control.xls")
        wget.download("https://raw.githubusercontent.com/smithandrewk/sleep/main/data/control.xls","data/control.xls")
    if(not path.exists("data/deprivation.xls")):
        print("Downloading deprivation.xls")
        wget.download("https://raw.githubusercontent.com/smithandrewk/sleep/main/data/deprivation.xls","data/deprivation.xls")
    if(not path.exists("scripts")):
        print("Making Directory scripts")
        os.mkdir("scripts")   
    if(not path.exists("scripts/utils.py")):
        print("Downloading utils.py")
        wget.download("https://raw.githubusercontent.com/smithandrewk/sleep/main/scripts/utils.py","scripts/utils.py")
except ImportError:
    print("wget import does not exist, using python2.7 wget")
    if(not path.exists("data")):
        print("Making Directory data")
        !mkdir data   
    if(not path.exists("data/control.xls")):
        print("control.xls not in data")
        !wget -O data/control.xls https://raw.githubusercontent.com/smithandrewk/sleep/main/data/control.xls
    if(not path.exists("data/deprivation.xls")):
        print("deprivation.xls not in data")
        !wget -O data/deprivation.xls https://raw.githubusercontent.com/smithandrewk/sleep/main/data/deprivation.xls
    if(not path.exists("scripts")):
        print("Making Directory scripts")
        !mkdir scripts
    if(not path.exists("scripts/utils.py")):
        !wget -O scripts/utils.py https://raw.githubusercontent.com/smithandrewk/sleep/main/scripts/utils.py

In [ ]:
from scripts.utils import *
df = preprocess("control")
## Statistics
# df.describe() # nice for obtaining statistics over dataframe
p,s,w = np.bincount(df['Class'])
total = p + s + w
print('Examples:\n    Total: {}\n    P: {} ({:.2f}% of total)\n    S: {} ({:.2f}% of total)\n    W: {} ({:.2f}% of total)\n'.format(
    total, p, 100 * p / total,s,100 * s / total,w,100 * w / total))

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
df = pd.read_csv("data/control_preprocessed.csv")

# Use a utility from sklearn to split and shuffle our dataset.
train_df, test_df = train_test_split(df, test_size=0.2)
# train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Class'))
p_train_labels = train_labels == 0
s_train_labels = train_labels == 1
w_train_labels = train_labels == 2

# val_labels = np.array(val_df.pop('Class'))
test_labels = np.array(test_df.pop('Class'))

train_features = np.array(train_df)
# val_features = np.array(val_df)
test_features = np.array(test_df)

In [ ]:
import tensorflow as tf
from tensorflow import keras
def get_compiled_model(n,dropout=True):
    """
    Function to create model. This is a sequential model, meaning layers execute
    one after the other. We have an input shape corresponding to the feature set,
    one hidden layer with 10 neurons and a relu activation, then an output layer
    with 3 neurons and a sigmoid activation function. We compute loss with
    categorical crossentropy and optimize with adam, which I believe is something
    about an adaptive learning rate. I do not know what the parameter from_logits
    is about.
    """
    if(dropout):
        model = tf.keras.Sequential([
        keras.layers.Dense(n, activation='relu',input_shape=(train_features.shape[-1],)),
        keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(3, activation='sigmoid')
        ])
    else:
        model = tf.keras.Sequential([
        keras.layers.Dense(n, activation='relu',input_shape=(train_features.shape[-1],)),
        tf.keras.layers.Dense(3, activation='sigmoid')
        ])
    model.compile(optimizer=keras.optimizers.Adam(lr=1e-3),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model
EPOCHS = 100
BATCH_SIZE = 200
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='accuracy', 
    verbose=1,
    patience=100,
    mode='max',
    restore_best_weights=True)

In [ ]:
model = get_compiled_model(20,dropout=False)
"""
We one-hot encode the targets. Mathematically, this is good for calculating
loss. CategoricalCrossEntropy simplifies to a negative log when targets are
one-hot encoded. However, I simply recieved an error from model.fit when I 
did not one-hot encode.
  @y : targets
  @depth : number of targets
"""
baseline_history = model.fit(
    train_features,
    tf.one_hot(train_labels,depth=3),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping])

In [ ]:
train_predictions_baseline = model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_baseline = model.predict(test_features, batch_size=BATCH_SIZE)

baseline_results = model.evaluate(test_features, tf.one_hot(test_labels,depth=3),
                                  batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, baseline_results):
  print(name, ': ', value)
print()

plot_cm(tf.one_hot(test_labels,depth=3).numpy().argmax(axis=1), test_predictions_baseline.argmax(axis=1))